In [35]:
import pandas as pd
from scipy.stats import chi2_contingency
import re

In [44]:
data = pd.read_csv('wanted_result.csv', index_col = 0)
data.drop_duplicates(inplace = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1850 entries, 0 to 1961
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   기업명            1850 non-null   object 
 1   업종분류           1850 non-null   object 
 2   사원수            928 non-null    float64
 3   설립연도           1850 non-null   int64  
 4   기업형태           1050 non-null   object 
 5   매출액            1693 non-null   float64
 6   주소             1850 non-null   object 
 7   직무             1850 non-null   object 
 8   자격요건           1832 non-null   object 
 9   직급             1850 non-null   int64  
 10  이용하는기술스택/우대사항  1844 non-null   object 
 11  해당 페이지 URL     1850 non-null   object 
 12  기업 홈페이지 URL    1829 non-null   object 
 13  수요             1850 non-null   int64  
dtypes: float64(2), int64(3), object(9)
memory usage: 216.8+ KB


In [49]:
contingency_table = pd.crosstab(data['수요'], data['직무'])
display(contingency_table)
chi2 , p_value , _, _= chi2_contingency(contingency_table)
print('Chi-square statistic:', chi2)
print('P-value:', p_value)
alpha = 0.05

if p_value < alpha:
    print("귀무가설 기각. \n통계적으로 유의미한 연관성이 있다.")
else:
    print("귀무가설 기각할 수 없음. \n통계적으로 유의미한 연관성이 없다.")

직무,.NET 개발자,BI 엔지니어,"C,C++ 개발자","CIO,Chief Information Officer","CTO,Chief Technology Officer",DBA,DevOps / 시스템 관리자,ERP전문가,Node.js 개발자,PHP 개발자,...,"영상,음성 엔지니어",웹 개발자,웹 퍼블리셔,임베디드 개발자,자바 개발자,크로스플랫폼 앱 개발자,파이썬 개발자,프로덕트 매니저,프론트엔드 개발자,하드웨어 엔지니어
수요,,,,,,,,,,,,,,,,,,,,,
0,5,0,4,0,1,0,6,1,7,2,...,1,26,1,2,17,2,10,2,20,1
1,6,3,35,1,9,12,76,5,72,8,...,10,222,12,31,135,18,70,22,188,17


Chi-square statistic: 43.239401440277874
P-value: 0.1598697027655848
귀무가설 기각할 수 없음. 
통계적으로 유의미한 연관성이 없다.


# 업종 분류별 수요가 높은 그룹의 특징

* 주소 
>Chi-square statistic: 346.1432727026265                 
>P-value: 1.339987132608967e-39                                 
>귀무가설 기각.                                      
>통계적으로 유의미한 연관성이 있다.

* 직급
>Chi-square statistic: 18.808470193453104                            
>P-value: 0.042763931049652294                                      
>귀무가설 기각.                                     
>통계적으로 유의미한 연관성이 있다.

* 사원수
>Chi-square statistic: 400.0730361118119                           
>P-value: 3.8525803736915116e-35     
>귀무가설 기각.         
>통계적으로 유의미한 연관성이 있다.

* 직무 
>Chi-square statistic: 43.239401440277874          
>P-value: 0.1598697027655848                  
>귀무가설 기각할 수 없음.                      
>통계적으로 유의미한 연관성이 없다.